## using 100A/50ma CT's

#### 1. Calculate Burden Resistor ($R_{burden}$)
Goal: Generate ~1.6V Peak at 100A (Max capacity of CT).

*   **CT Ratio**: 100A : 0.050A (50mA)
*   **Secondary Max RMS**: $I_{rms} = 0.050 A$
*   **Secondary Max Peak**: $I_{peak} = I_{rms} \times \sqrt{2} = 0.050 \times 1.414 = 0.0707 A$
*   **Target Voltage**: 1.62V Peak
*   **Resistor Value**:
    $$ R_{burden} = \frac{1.62V}{0.0707A} = 22.9 \Omega $$
**Selection**: A standard **22 Ohm** or **24 Ohm** resistor is perfect.
*   Peak Voltage at 100A (22$\Omega$) = $1.55V$
*   Power Dissipation: $P = I^2 R = 0.05^2 \times 22 = 0.055 W$ (55mW).
*   **Result**: A standard tiny 1/4W resistor works safely.

#### 2. Wiring the Plug (3.5mm)
*   **Resistor**: Connect one leg to CT Red, other leg to CT Black.
*   **Tip**: Connect **Red** wire to the Tip of the plug.
*   **Sleeve**: Connect **Black** wire to the Sleeve (Ground) of the plug.
*   **Ring**: Connect to Sleeve (or leave disconnected).

## WIFI

#### lib deps
https://github.com/tzapu/WiFiManager.git is tzapu/WiFiManager@^2.0.6 
#### SSID
"CT_Monitor_Setup" is the SSID, 192.168.4.1 is the web address

### WIFI minimal code

The "modern" way to handle WiFi on ESP32/ESP8266 without hardcoding credentials is to use a library that manages the **Captive Portal** experience.

The best standard library for this is **WiFiManager** (by tzapu).
*   **How it works**: When the ESP starts, it tries to connect to the last known WiFi. If it fails (or has no credentials), it turns into an Access Point (AP) named "ESP32_Config" (or similar). You connect to it with your phone, a webpage pops up, and you enter your home WiFi credentials. It then saves them and reboots.
*   **Storage**: It handles the saving to NVS (Non-Volatile Storage) automatically so you don't need to write file system code.

Here is how to implement it cleanly in `connWIFI.h` and `connWIFI.cpp`.

### 1. Update `platformio.ini`
Add the WiFiManager dependency.

```ini
lib_deps =
    adafruit/Adafruit ADS1X15@^2.6.0
    https://github.com/tzapu/WiFiManager.git
```

### 2. Create `src/connWIFI.h`
We only need a simple setup function.

```cpp
#ifndef connWIFI_h
#define connWIFI_h

#include <Arduino.h>

// This function will block until connected or timeout
bool setupWIFI();

// Optional: Function to reset settings (for testing/debugging)
void resetWIFI();

#endif
```

### 3. Create `src/connWIFI.cpp`
This wraps the library implementation.

```cpp
#include <WiFiManager.h> // https://github.com/tzapu/WiFiManager
#include "connWIFI.h"

bool setupWIFI() {
    WiFiManager wm;
    // Optional: Add debug output
    // wm.setDebugOutput(true);
    // Optional: Set a timeout (in seconds) so it doesn't hang forever in AP mode
    // if configuration fails. If it times out, it returns false.
    wm.setConfigPortalTimeout(180); // 3 minutes to configure
    // Automatically connect using saved credentials,
    // if connection fails, it starts an access point with the specified name
    // "CT_Monitor_Setup" is the SSID, 192.168.4.1 is the web address
    bool res = wm.autoConnect("CT_Monitor_Setup"); // AP Name

    if(!res) {
        Serial.println("Failed to connect or hit timeout");
        return false;
    } 
    
    Serial.println("WiFi Connected... yeey :)");
    Serial.print("IP Address: ");
    Serial.println(WiFi.localIP());
    return true;
}

void resetWIFI() {
    WiFiManager wm;
    wm.resetSettings();
    Serial.println("WiFi settings reset");
}
```

### 4. Update `src/main.cpp`
Call it in your setup.

```cpp
#include "connWIFI.h"

void setup() {
    Serial.begin(115200);
    // ... other setup ...

    Serial.println("Initializing WiFi...");
    if (!setupWIFI()) {
        Serial.println("Critical Error: WiFi connection failed. Restarting...");
        ESP.restart();
    }
    
    // ... continue with setupCT() ...
}
```

## separation of concerns
*   **Separation of Concerns**: `CT_Config` is for settings that you determine *before* compiling (What pin? What gain? What threshold?). `CT_Runtime` is for values that change *while running* (Current amps, Last reported amps).
*   
### in ct.h (The Blueprints)
```cpp
struct CT_Config {
  uint8_t   pin;
  adsGain_t gain;
  float     lsbVolts;
  float     m;
  float     b;
  int       capacity;
  float     threshold;  // <--- Add here. E.g. 0.5A
  const char* name;
};
```

**2. Update `CT_Runtime` (The Memory)**
Keep this strictly for tracking state.

```cpp
struct CT_Runtime {
  float currentValue;
  float lastReportedValue; 
};

### in CONFIG.cpp (the actual data)

```cpp
CT_Config sensors[4] = {
  {0, GAIN_ONE, 0.0001875,  104.7, 0.922, 15, .3, "Heat_Pump"},
  {1, GAIN_ONE, 0.0001875,  104.7, 0.922, 15, .3, "Solar"},
  {2, GAIN_ONE, 0.0001875,  104.7, 0.922, 50, .3, "EV"},
  {3, GAIN_ONE, 0.0001875,  104.7, 0.922, 15, .3, "empty"},
};
```

### in ct4.cpp (the  logic (functions))

### in main.cpp (the memory)
The **Runtime State** lives here because it belongs to the `loop()`.

```cpp
// Global State Array
CT_Runtime ctState[4]; // All zeros by default

void loop() {


# to have vscode use conda env:ai
in .vscode/settings.json in both ct4 and ct4/esp-ct4 folders add
```json
{
    "python.defaultInterpreterPath": "C:\\Users\\mcken\\miniconda3\\envs\\ai\\python.exe",
    "terminal.integrated.defaultProfile.windows": "Git Bash"
}
```



## esp memory
By default, the ESP32 partition map is split to allow for "Over-the-Air" (OTA) updates. It reserves half the space for the running app (App0) and half for the update (App1), roughly **1.25MB** each.

Since your hardware has **4MB** of Flash, you can simply change the **Partition Scheme** in `platformio.ini` to allocate more space to your application and less to other features (like SPIFFS or OTA) if you need it.

#### Solution: Change Partition Scheme

You can add `board_build.partitions` to your `[env:esp32doit-devkit-v1]` section in `platformio.ini`.

**Option 1: "Huge App" (No OTA updates)**
If you always update via USB cable, use this to get almost 3MB for your code.
```ini
[env:esp32doit-devkit-v1]
; ... existing settings ...
board_build.partitions = huge_app.csv
```

**Option 2: "Min SPIFFS" (With OTA updates)**
If you need OTA updates but don't store many files on the ESP32 file system.
```ini
[env:esp32doit-devkit-v1]
; ... existing settings ...
board_build.partitions = min_spiffs.csv
```

#### Recommendation
For now, since you are in development, switching to **huge_app.csv** is the easiest fix to remove anxiety about hitting the ceiling.


##n